<a href="https://colab.research.google.com/github/DurgaBhavana/5731Submissions/blob/master/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 11/11/2020)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/INFO5731_FALL2020/blob/master/In_class_exercise/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [212]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [228]:
import warnings
warnings.filterwarnings('ignore')

In [163]:
def reading_data(file_path):
  text_data, sentiments = [], []
  file_data = open(file_path).read()
  for i, j in enumerate(file_data.split("\n")):
    after_split = j.split(' ')
    text_data.append(" ".join(after_split[1:]))
    sentiments.append(after_split[0])
  return text_data, sentiments

In [170]:
training_text_data, training_sentiments = reading_data('/content/stsa-train.txt')
training_df = pd.DataFrame(list(zip(training_sentiments, training_text_data)), columns = ['Sentimental Value', 'Raw Data'])
testing_text_data, testing_sentiments = reading_data('/content/stsa-test.txt')
testing_df = pd.DataFrame(list(zip(testing_sentiments, testing_text_data)), columns = ['Sentimental Value', 'Raw Data'])

In [181]:
#Pre-processing
# Special characters removal
training_df['After noise removal'] = training_df['Raw Data'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
testing_df['After noise removal'] = testing_df['Raw Data'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
# Punctuation removal
training_df['Punctuation removal'] = training_df['After noise removal'].str.replace('[^\w\s]','')
testing_df['Punctuation removal'] = testing_df['After noise removal'].str.replace('[^\w\s]','')
# Stopwords removal
stop_word = stopwords.words('english')
training_df['Stopwords removal'] = training_df['Punctuation removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))
testing_df['Stopwords removal'] = testing_df['Punctuation removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_word))
# Lower Casing
training_df['Lower casing'] = training_df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))
testing_df['Lower casing'] = testing_df['Stopwords removal'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [184]:
tfidf_vector = TfidfVectorizer(analyzer = 'word')
tfidf_vector.fit(training_df['Lower casing'])
x =  tfidf_vector.transform(training_df['Lower casing'])
tfidf_vector_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vector.vocabulary_)
tfidf_vector_test.fit(testing_df['Lower casing'])
test_values_x = tfidf_vector_test.transform(testing_df['Lower casing'])

In [193]:
xtrain, xvalid, ytrain, yvalid = model_selection.train_test_split(x, training_df['Sentimental Value'],test_size=0.2)

In [189]:
def csv(model, x_data, y_data):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7, shuffle=True)
  return cross_val_score(model, x_data, y_data, cv=kfold).mean()

In [224]:
def training_model(model_intializer):
  model = model_intializer
  model.fit(xtrain, ytrain)
  predicted = model.predict(xvalid)
  accuracy = accuracy_score(predicted, yvalid)
  print("Accuracy of Traning data: {0}".format(accuracy))
  print(classification_report(yvalid, predicted))
  predicted_testing = model.predict(test_values_x)
  accuracy_testing = accuracy_score(predicted_testing, testing_df['Sentimental Value'])
  print("Accuracy of Testing data: {0}".format(accuracy_testing))
  print(classification_report(testing_df['Sentimental Value'], predicted_testing))
  if 'XGB' not in str(model):
    print("Cross validation score obtained: {0}".format(csv(model, test_values_x, testing_df['Sentimental Value'])))

In [229]:
training_model(naive_bayes.MultinomialNB())

Accuracy of Traning data: 0.7927797833935019
              precision    recall  f1-score   support

           0       0.79      0.74      0.77       637
           1       0.79      0.83      0.81       748

    accuracy                           0.79      1385
   macro avg       0.79      0.79      0.79      1385
weighted avg       0.79      0.79      0.79      1385

Accuracy of Testing data: 0.7925356750823271
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.85      0.71      0.77       912
           1       0.75      0.87      0.81       909

    accuracy                           0.79      1822
   macro avg       0.53      0.53      0.53      1822
weighted avg       0.80      0.79      0.79      1822

Cross validation score obtained: 0.7365309553834145


In [230]:
training_model(svm.SVC())

Accuracy of Traning data: 0.7956678700361011
              precision    recall  f1-score   support

           0       0.77      0.80      0.78       637
           1       0.82      0.79      0.81       748

    accuracy                           0.80      1385
   macro avg       0.79      0.80      0.79      1385
weighted avg       0.80      0.80      0.80      1385

Accuracy of Testing data: 0.7886937431394072
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.81      0.75      0.78       912
           1       0.77      0.83      0.80       909

    accuracy                           0.79      1822
   macro avg       0.53      0.53      0.53      1822
weighted avg       0.79      0.79      0.79      1822

Cross validation score obtained: 0.7217017954722873


In [231]:
training_model(KNeighborsClassifier(n_neighbors = 5))

Accuracy of Traning data: 0.4996389891696751
              precision    recall  f1-score   support

           0       0.47      0.71      0.57       637
           1       0.57      0.32      0.41       748

    accuracy                           0.50      1385
   macro avg       0.52      0.52      0.49      1385
weighted avg       0.52      0.50      0.48      1385

Accuracy of Testing data: 0.5005488474204172
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.50      0.71      0.59       912
           1       0.50      0.29      0.37       909

    accuracy                           0.50      1822
   macro avg       0.33      0.33      0.32      1822
weighted avg       0.50      0.50      0.48      1822

Cross validation score obtained: 0.5005464480874318


In [232]:
training_model(DecisionTreeClassifier())

Accuracy of Traning data: 0.6425992779783394
              precision    recall  f1-score   support

           0       0.60      0.68      0.64       637
           1       0.69      0.61      0.65       748

    accuracy                           0.64      1385
   macro avg       0.65      0.65      0.64      1385
weighted avg       0.65      0.64      0.64      1385

Accuracy of Testing data: 0.6761800219538968
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.67      0.68      0.68       912
           1       0.68      0.67      0.67       909

    accuracy                           0.68      1822
   macro avg       0.45      0.45      0.45      1822
weighted avg       0.68      0.68      0.68      1822

Cross validation score obtained: 0.6031826097399866


In [233]:
training_model(RandomForestClassifier())

Accuracy of Traning data: 0.7357400722021661
              precision    recall  f1-score   support

           0       0.69      0.77      0.73       637
           1       0.78      0.70      0.74       748

    accuracy                           0.74      1385
   macro avg       0.74      0.74      0.74      1385
weighted avg       0.74      0.74      0.74      1385

Accuracy of Testing data: 0.7420417124039517
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.73      0.76      0.75       912
           1       0.75      0.72      0.74       909

    accuracy                           0.74      1822
   macro avg       0.49      0.49      0.49      1822
weighted avg       0.74      0.74      0.74      1822

Cross validation score obtained: 0.6575331772053084


In [234]:
training_model(XGBClassifier())
scoring = 'accuracy'
kfold = KFold(10, random_state = 7, shuffle=True)
print("Cross validation score obtained: {0}".format(cross_val_score(XGBClassifier(), test_values_x, testing_df['Sentimental Value']).mean()))

Accuracy of Traning data: 0.6418772563176895
              precision    recall  f1-score   support

           0       0.69      0.40      0.51       637
           1       0.62      0.85      0.72       748

    accuracy                           0.64      1385
   macro avg       0.66      0.62      0.61      1385
weighted avg       0.65      0.64      0.62      1385

Accuracy of Testing data: 0.6487376509330406
              precision    recall  f1-score   support

                   0.00      0.00      0.00         1
           0       0.75      0.45      0.56       912
           1       0.60      0.85      0.71       909

    accuracy                           0.65      1822
   macro avg       0.45      0.43      0.42      1822
weighted avg       0.68      0.65      0.63      1822

Cross validation score obtained: 0.6059220231822973
